# **End Phase 2 Project**

In [44]:
# importing necessary libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import statistics

import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

from scipy.stats import ttest_ind, ttest_1samp, chi2_contingency, norm

from sklearn.preprocessing import StandardScaler, LabelEncoder

import os
import pickle

import warnings
warnings.filterwarnings('ignore')

## Data Loading  

We begin by loading all datasets:
- SQLite database (`im.db`) containing IMDb-like tables.  
- CSV/TSV files providing box office, TMDB, Rotten Tomatoes, and budget information.  

These diverse sources will later be merged into a unified dataset for analysis.


In [45]:
import sqlite3
conn = sqlite3.connect(r"C:\Users\PC\Desktop\Group_3_Project\im.db")
cursor = conn.cursor()

tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


# Loading Relevant Tables and Data

In [46]:
# Load main tables from the database
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics;", conn)
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings;", conn)

# Load CSV and TSV files
movie_gross = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\bom.movie_gross.csv")
tmdb = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\tmdb.movies.csv")
movie_budgets = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\tn.movie_budgets.csv")
rt_movie = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\rt.movie_info.tsv", sep='\t')
reviews = pd.read_csv(r"C:\Users\PC\Documents\Phase-2-project-G3\Data\rt.reviews.tsv", sep='\t', encoding='latin1')


**Checking basic info from the tables**

In [47]:
movie_basics = pd.read_sql_query("SELECT * FROM movie_basics LIMIT 5;", conn)
movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [48]:
movie_ratings = pd.read_sql_query("SELECT * FROM movie_ratings LIMIT 5;", conn)
movie_ratings

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [49]:
movie_gross.head(2)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [50]:
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [51]:
tmdb.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [52]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [53]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [54]:
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [55]:
rt_movie.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [56]:
rt_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [57]:
reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [58]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


# **Data Cleaning Preprocessing**

Before analysis, we clean all datasets to ensure consistency and usability.  
Since these datasets come from different sources, they vary in completeness and format.  
Each table is cleaned individually, guided by these principles:  

1. Retain at least 70% of the data after cleaning.  
2. Handle missing values through imputation or careful removal.  
3. Convert data types (dates, numeric columns).  
4. Standardize text for consistent merging later.  

SQLite tables:  

            *movie_basics*

            *movie_ratings*


In [59]:
movie_basics.isna().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    1
genres             0
dtype: int64

Only 1 missing value in "runtime_minutes".  
Fill missing runtime with the median runtime (since runtime is numeric and often right-skewed).  

In [60]:
# Fill missing runtime_minutes with median
median_runtime = movie_basics['runtime_minutes'].median()
movie_basics['runtime_minutes'].fillna(median_runtime, inplace=True)                              


In [61]:
movie_basics.isna().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
dtype: int64

### Cleaning `movie_ratings`

- All columns (`movie_id`, `averagerating`, `numvotes`) have **no missing values**.  
- No cleaning required for this table


In [62]:
movie_ratings.isna().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

### Cleaning `movie_gross`

In [63]:
movie_gross.isnull().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

### Cleaning `movie_gross`

**Cleaning Strategy:**
- Drop rows missing both domestic and foreign gross.
- Convert `foreign_gross` to numeric (some are stored as strings).  
- Fill remaining `foreign_gross` with **median foreign gross** to retain ≥70% of rows.  


In [64]:
# Convert foreign_gross to numeric
movie_gross['foreign_gross'] = pd.to_numeric(movie_gross['foreign_gross'], errors='coerce')

# Drop rows missing both gross values
movie_gross = movie_gross.dropna(subset=['domestic_gross', 'foreign_gross'], how='all')

# Fill missing foreign_gross with median (preserve 70%+)
median_foreign = movie_gross['foreign_gross'].median()
movie_gross['foreign_gross'].fillna(median_foreign, inplace=True)

# Fill missing studio with "Unknown"
movie_gross['studio'].fillna("Unknown", inplace=True)

In [65]:
# Confirming nulls
movie_gross.isna().sum()

title              0
studio             0
domestic_gross    28
foreign_gross      0
year               0
dtype: int64

In [66]:
# droping null rows in domestic gross

movie_gross = movie_gross.dropna(subset=['domestic_gross'])

In [67]:
# Confirming nulls
movie_gross.isna().sum()

title             0
studio            0
domestic_gross    0
foreign_gross     0
year              0
dtype: int64

### Cleaning `tmdb`

In [68]:
tmdb.isnull().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

This dataset is complete — no missing values detected in any of the 10 columns.  

No action required.

converting the `release_date` column to a proper datetime for easier year extraction.


In [69]:
tmdb['release_date'] = pd.to_datetime(tmdb['release_date'], errors='coerce')
tmdb['release_year'] = tmdb['release_date'].dt.year

tmdb.head(3)


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,release_year
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,2010
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,2010
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,2010


### Cleaning `movie_budgets`

In [70]:
movie_budgets.isnull().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

This table has no missing values.
However, monetary values (`production_budget`, `domestic_gross`, `worldwide_gross`) are stored as strings with `$` and commas.  
converting them to numeric to support analysis and merging.


In [71]:
def parse_money(x):
    if pd.isna(x):
        return np.nan
    return float(str(x).replace('$', '').replace(',', ''))

movie_budgets['production_budget'] = movie_budgets['production_budget'].apply(parse_money)
movie_budgets['domestic_gross'] = movie_budgets['domestic_gross'].apply(parse_money)
movie_budgets['worldwide_gross'] = movie_budgets['worldwide_gross'].apply(parse_money)

movie_budgets.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5782 non-null   int64  
 1   release_date       5782 non-null   object 
 2   movie              5782 non-null   object 
 3   production_budget  5782 non-null   float64
 4   domestic_gross     5782 non-null   float64
 5   worldwide_gross    5782 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 271.2+ KB


### Cleaning `rt_movie`

In [72]:
rt_movie.isnull().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

This dataset has several columns with high null percentages: 

In [73]:
# Droping highly incomplete columns
rt_movie.drop(columns=['currency', 'box_office'], inplace=True)


In [74]:
# Filling textual columns

rt_movie['synopsis'].fillna('Not available', inplace=True)
rt_movie['rating'].fillna(method='ffill', inplace=True)
rt_movie['genre'].fillna('Unknown', inplace=True)
rt_movie['director'].fillna('Unknown', inplace=True)
rt_movie['writer'].fillna('Unknown', inplace=True)
rt_movie['studio'].fillna('Independent', inplace=True)

In [75]:
# Filling runtime with mode

mode_runtime = rt_movie['runtime'].mode()[0]
rt_movie['runtime'].fillna(mode_runtime, inplace=True)

In [76]:
# confirming no nulls remain

rt_movie.isna().sum()

id                0
synopsis          0
rating            0
genre             0
director          0
writer            0
theater_date    359
dvd_date        359
runtime           0
studio            0
dtype: int64

In [77]:
# droping null rows in theater_date and dvd_date

rt_movie = rt_movie.dropna(subset=['theater_date', 'dvd_date'])

In [78]:
rt_movie.isna().sum()

id              0
synopsis        0
rating          0
genre           0
director        0
writer          0
theater_date    0
dvd_date        0
runtime         0
studio          0
dtype: int64